## 原始数据统计分析

### 文本长度分布

In [1]:
import pandas as pd
len_list = []
file_list = ['test', 'train', 'val']
for file in file_list:
    with open(f'../data/fold0/{file}/sentences.txt', 'r', encoding='utf-8') as f:
        len_list += [len(line.strip().split(' ')) for line in f]
print(pd.Series(len_list).describe(percentiles=[.25, .5, .75, .85, .95]))

count    800.000000
mean     149.696250
std       44.085552
min        2.000000
25%      120.750000
50%      145.000000
75%      175.000000
85%      191.000000
95%      228.050000
max      358.000000
dtype: float64


### 分类别实体分布

In [2]:
import sys
sys.path.insert(0, '../')
from metrics import get_entities
from utils import IO2STR

In [7]:
data_dir = ['../data/fold0/val/', '../data/fold0/train/']
# data_dir = ['../data/fold0/test/']

In [8]:
sentences = []
tags = []
for dir_ in data_dir:
    with open(dir_+'sentences.txt', 'r', encoding='utf-8') as f_sen,\
        open(dir_+'tags.txt', 'r', encoding='utf-8') as f_tag:
        sentences.extend([line.strip().split(' ') for line in f_sen])
        tags.extend([line.strip().split(' ') for line in f_tag])

In [9]:
y_true = []
for s,t in zip(sentences, tags):
    entities = get_entities(t)
    sample = []
    for entity in entities:
        entity_con = s[entity[1]:entity[2]+1]
        entity_type = IO2STR[entity[0]]
        sample.append((entity_type, ''.join(entity_con)))
    y_true.extend(sample) 

In [10]:
y_true

[('试验要素', '“加尔各答”级驱逐舰'),
 ('试验要素', '远程面空导弹'),
 ('试验要素', '增程型空中目标'),
 ('试验要素', '“巴拉克”-8导弹'),
 ('试验要素', '波音KC-46A加油机'),
 ('试验要素', '空中加油试验'),
 ('性能指标', '飞行高度'),
 ('试验要素', 'F-16战斗机'),
 ('试验要素', '燃油'),
 ('系统组成', '吊杆'),
 ('系统组成', '伸缩管'),
 ('系统组成', '插孔'),
 ('试验要素', '受油机'),
 ('试验要素', '无人潜航器'),
 ('试验要素', '通用控制系统'),
 ('试验要素', '软件架构'),
 ('系统组成', '框架'),
 ('系统组成', '用户界面'),
 ('系统组成', '元件'),
 ('任务场景', '无人系统'),
 ('任务场景', '海军无人系统'),
 ('任务场景', '航行器管理'),
 ('任务场景', '任务计划'),
 ('任务场景', '任务管理能力'),
 ('系统组成', '第一级'),
 ('系统组成', '主发动机'),
 ('试验要素', '“南半球运载火箭”项目'),
 ('试验要素', '方案'),
 ('试验要素', '飞行试验'),
 ('系统组成', '带翼助推器'),
 ('系统组成', '超燃冲压发动机'),
 ('系统组成', '第二级'),
 ('试验要素', '“地基中段防御系统”'),
 ('试验要素', '非拦截飞行试验'),
 ('试验要素', '“能力增强2”型大气层外杀伤拦截器'),
 ('系统组成', '助推器'),
 ('试验要素', '“能力增强2”型地基拦截弹'),
 ('系统组成', 'EKV'),
 ('系统组成', '质量配比'),
 ('系统组成', '惯性测量单元'),
 ('试验要素', '“皮那卡”Ⅱ多管火箭炮发射系统'),
 ('试验要素', '用户辅助试验'),
 ('试验要素', '“皮那卡”Ⅰ武器系统'),
 ('性能指标', '射程'),
 ('系统组成', '子母弹弹头'),
 ('任务场景', '印度陆军'),
 ('任务场景', '俄制BM-21“冰雹”发射器'),
 ('试验要素', '坦克'),


In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame({
    "type": [ne[0] for ne in y_true],
    "content": [ne[1] for ne in y_true]
})

In [13]:
df['length'] = df.apply(lambda row: len(row.content), axis=1)

In [14]:
df

,type,content,length
0,试验要素,“加尔各答”级驱逐舰,10
1,试验要素,远程面空导弹,6
2,试验要素,增程型空中目标,7
3,试验要素,“巴拉克”-8导弹,9
4,试验要素,波音KC-46A加油机,11
...,...,...,...
3156,试验要素,高空“伪卫星”无人机,10
3157,性能指标,有效载荷,4
3158,系统组成,通信中继器,5
3159,系统组成,拍照录像设备,6


In [16]:
# 按类别分组
df_gb = df.groupby('type')

### 训练集实体出现次数

In [87]:
from collections import Counter

In [88]:
with open('./val_freq.txt', 'w', encoding='utf-8') as f:
    for entity, freq in Counter(y_true).most_common():
        f.write(f'{entity[0]}\t{entity[1]}\t{freq}\n')

### 分类别实体个数

In [17]:
df_gb.count()

,content,length
type,,
任务场景,516,516
性能指标,712,712
系统组成,396,396
试验要素,1537,1537


### 分类别实体长度

In [18]:
from collections import Counter

In [19]:
df_gb["length"].mean()

type
任务场景    5.106589
性能指标    3.817416
系统组成    5.207071
试验要素    8.554327
Name: length, dtype: float64

In [20]:
sorted(Counter(df.loc[df_gb.groups['任务场景']]['length'].tolist()).most_common(), key=lambda x: x[0])

[(1, 1),
 (2, 103),
 (3, 52),
 (4, 158),
 (5, 35),
 (6, 54),
 (7, 27),
 (8, 21),
 (9, 14),
 (10, 12),
 (11, 11),
 (12, 10),
 (13, 5),
 (14, 3),
 (15, 3),
 (16, 2),
 (17, 1),
 (18, 2),
 (19, 1),
 (41, 1)]

In [102]:
sorted(Counter(df.loc[df_gb.groups['性能指标']]['length'].tolist()).most_common(), key=lambda x: x[0])

[(1, 27),
 (2, 208),
 (3, 99),
 (4, 222),
 (5, 34),
 (6, 56),
 (7, 26),
 (8, 11),
 (9, 9),
 (10, 10),
 (11, 3),
 (12, 2),
 (13, 3),
 (14, 1),
 (23, 1)]

In [103]:
sorted(Counter(df.loc[df_gb.groups['系统组成']]['length'].tolist()).most_common(), key=lambda x: x[0])

[(1, 1),
 (2, 65),
 (3, 60),
 (4, 82),
 (5, 42),
 (6, 43),
 (7, 33),
 (8, 22),
 (9, 10),
 (10, 14),
 (11, 7),
 (12, 6),
 (13, 1),
 (14, 2),
 (15, 3),
 (16, 3),
 (17, 1),
 (18, 1)]

In [105]:
sorted(Counter(df.loc[df_gb.groups['试验要素']]['length'].tolist()).most_common(), key=lambda x: x[0])

[(1, 3),
 (2, 100),
 (3, 60),
 (4, 282),
 (5, 76),
 (6, 189),
 (7, 95),
 (8, 129),
 (9, 87),
 (10, 77),
 (11, 58),
 (12, 59),
 (13, 65),
 (14, 45),
 (15, 39),
 (16, 28),
 (17, 28),
 (18, 25),
 (19, 19),
 (20, 18),
 (21, 9),
 (22, 9),
 (23, 10),
 (24, 5),
 (25, 5),
 (26, 4),
 (27, 3),
 (28, 2),
 (29, 2),
 (30, 1),
 (33, 1),
 (35, 1),
 (36, 2),
 (41, 1)]

## Badcase分析

In [25]:
from pathlib import Path

In [26]:
val_data_dir = Path('../data/fold0/val/')

In [27]:
with open(val_data_dir/'sentences.txt', 'r', encoding='utf-8') as f_sen,\
    open(val_data_dir/'tags.txt', 'r', encoding='utf-8') as f_tag:
    sentences = [line.strip().split(' ') for line in f_sen]
    tags = [line.strip().split(' ') for line in f_tag]

获取gold label

In [28]:
y_true = []
for s,t in zip(sentences, tags):
    entities = get_entities(t)
    sample = []
    for entity in entities:
        entity_con = s[entity[1]:entity[2]+1]
        entity_type = IO2STR[entity[0]]
        sample.append((entity_type, ''.join(entity_con), entity[1], entity[2]+1))
    y_true.append(sample) 

获取pre label

In [32]:
ex1_dir = Path('./val_result/fusion_val.json')

In [33]:
import json

In [34]:
with open(ex1_dir, 'r', encoding='utf-8') as f:
    pre = json.load(f)

In [36]:
y_pre = []
for idx in range(1, len(pre)+1):
    text = ''.join(sentences[idx-1])
    entities = pre[f'validate_V2_{idx}.json']
    sample = []
    for entity in entities:
        entity_con = text[entity['start_pos']-1:entity['end_pos']]
        sample.append((entity['label_type'], entity_con, entity['start_pos']-1, entity['end_pos']))
    y_pre.append(sample)

In [39]:
for sen, pre, true in zip(sentences, y_pre, y_true):
        print("".join(sen))
        print('-*-'*40)
        # 真实标签里有而预测结果里没有的（影响Recall）
        print(sorted(list(set(true).difference(set(pre))), key=lambda p: p[2]))
        print('-*-'*40)
        # 预测结果里有而真实标签里没有的（影响Precision）
        print(sorted(list(set(pre).difference(set(true))), key=lambda p: p[2]))
        print('-*-'*40)
        print('true_label:', sorted(true, key=lambda p: p[2]))
        print('-*-'*40)
        print('pre_label:', sorted(pre, key=lambda p: p[2]))
        print('\n')

印度海军近日从“加尔各答”级驱逐舰上试射了由印度和以色列联合研制的远程面空导弹?试验在位于印度西海岸的“加尔各答”级驱逐舰上进行?试验中,该导弹成功拦截了一个增程型空中目标?此次试验的“巴拉克”-8导弹由印度国防研究与发展局?以色列航空工业公司?以色列武器研发与技术基础设施管理局?埃尔塔系统公司?拉法尔公司等机构联合研制?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“巴拉克”-8导弹', 92, 101)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '“加尔各答”级驱逐舰', 7, 17), ('试验要素', '远程面空导弹', 33, 39), ('试验要素', '增程型空中目标', 79, 86), ('试验要素', '“巴拉克”-8导弹', 92, 101)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '“加尔各答”级驱逐舰', 7, 17), ('试验要素', '远程面空导弹', 33, 39), ('试验要素', '增程型空中目标', 79, 86)]


美国空军24日在华盛顿州成功完成波音KC-46A加油机(工程与制造第2阶段)首次空中加油试验,开启了空中加油演示验证的里程碑C阶段?作为开发的一部分,试验小组完成了

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('任务场景', '俄', 136, 137), ('试验要素', 'T55坦克', 138, 143)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '坦克', 15, 17), ('试验要素', '热压弹药', 25, 29), ('试验要素', '“尘雾射击”试验', 32, 40), ('试验要素', '“阿琼”主战坦克', 67, 75), ('试验要素', '高爆弹药', 106, 110), ('试验要素', '俄制T55坦克目标', 136, 145)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '坦克', 15, 17), ('试验要素', '热压弹药', 25, 29), ('试验要素', '“尘雾射击”试验', 32, 40), ('试验要素', '“阿琼”主战坦克', 67, 75), ('试验要素', '高爆弹药', 106, 110), ('任务场景', '俄', 136, 137), ('试验要素', 'T55坦克', 138, 143)]


美国海军研究实验室与宾夕法尼亚州立大学飞行器智能和自主实验室成功演示验证了2驾无人滑翔机在飞行中的数据共享?试验中,2驾无人滑翔机在1000多米的高空自主飞行超过5小时,并在飞行中协作?共享遥测数据?本次试验表明,2个实验室的自主飞行算法都能成功发现并利用上升热气流,并对协同飞行与共享信息?提高机群续航能力进行了概念验证?
-*--*--*--*--*--*--*--*--*--*

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹防御系统', 58, 66), ('试验要素', '分布式地面试验', 66, 73)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹防御系统分布式地面试验', 58, 73)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '弹道导弹防御系统', 58, 66), ('试验要素', '分布式地面试验', 66, 73), ('试验要素', '威胁想定', 80, 84), ('性能指标', '真实攻击', 91, 95)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '弹道导弹防御系统分布式地面试验', 58, 73), ('试验要素', '威胁想定', 80, 84), ('性能指标', '真实攻击', 91, 95)]


美国国防高级研究计划局近日在加利福尼亚海岸的开放水域对“反潜战持续跟踪无人艇”(ACTUV)搭载的“海上系统空中拖曳伞平台”(TALONS)样机进行了为期2天的飞行试验?试验中,TALONS样机从安装于ACTUV后部的“套”中脱离,随即张开翼伞并升至305米的高空,对其携带的传感器和通信设备进行试验?试验一结束,样机自动收回到“套”中?在整个试验过程中,ACTUV始终以实际工作速

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '三军联合演示验证', 10, 18), ('试验要素', '通用控制系统', 22, 28), ('试验要素', '无人直升机', 77, 82), ('试验要素', '地面车辆', 83, 87), ('试验要素', '伤亡撤离响应事件', 88, 96), ('试验要素', '联合工作', 105, 109), ('试验要素', '系统和标准', 110, 115), ('试验要素', '无人系统控制单元体系架构标准', 120, 134), ('任务场景', '空军', 136, 138), ('任务场景', '海军系统', 139, 143), ('试验要素', '任务协调', 146, 150)]


美国陆军“远征勇士实验”首次在实战演习中试验了“瞄准锁定”稳定武器平台,这种看上去颇为笨重的步枪可通过减少人为失误,提升普通步兵的平均作战效能?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '步枪', 46, 48)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '实战演习', 15, 19)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '“远征勇士实验”', 4, 12), ('试验要素', '“瞄准锁定”稳

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', 'BQM-177A亚声速飞行靶标', 29, 44), ('试验要素', 'SSAT项目', 86, 92), ('任务场景', '美国海军', 125, 129), ('任务场景', '亚声速反巡航导弹威胁', 135, 145), ('任务场景', '新型高逼真度靶标', 146, 154), ('任务场景', '舰队训练', 163, 167), ('任务场景', '武器系统', 168, 172)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', 'BQM-177A亚声速飞行靶标', 29, 44), ('试验要素', '飞行靶标与诱饵系统', 104, 113), ('任务场景', '美国海军', 125, 129), ('任务场景', '亚声速反巡航导弹威胁', 135, 145), ('任务场景', '新型高逼真度靶标', 146, 154), ('任务场景', '舰队训练', 163, 167), ('任务场景', '武器系统', 168, 172), ('任务场景', '试验鉴定', 173, 177)]


美国海军“约翰芬”号驱逐舰成功试射SM-2导弹?试验中,SM-2导弹分别从位于驱逐舰前方和后方的MK41垂直发射系统中发射?此次试验是“约翰芬”号驱逐舰交付海军前规划进行的3次试验中的第2次?第1次试验于2015年8月完成,第3次试验美国海军检查与调查委员会将参与其中?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('系统组成', '

...

[('任务场景', '全天候', 64, 67), ('性能指标', '飞行', 90, 92), ('性能指标', '速度', 92, 94), ('性能指标', '水面舰船', 106, 110), ('性能指标', '潜艇', 111, 113), ('性能指标', '飞机', 120, 122)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'RGM-84“鱼叉”', 46, 56), ('性能指标', '全天候', 64, 67), ('性能指标', '飞行速度', 90, 94), ('任务场景', '水面舰船', 106, 110), ('任务场景', '潜艇', 111, 113), ('任务场景', '飞机', 120, 122)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '“萨尔5”级护卫舰', 8, 17), ('试验要素', 'RGM-84“鱼叉”反舰导弹', 19, 33), ('试验要素', '新型武器', 36, 40), ('任务场景', '全天候', 64, 67), ('任务场景', '远距离反舰导弹', 68, 75), ('性能指标', '长', 76, 77), ('性能指标', '射程', 82, 84), ('性能指标', '飞行', 90, 92), ('性能指标', '速度', 92, 94), ('性能指标', '水面舰船', 106, 110), ('性能指标', '潜艇', 111, 113), ('性能指标', '飞机', 120, 122), ('任务场景', '出水潜艇', 132, 136), ('任务场景', '驱逐舰', 137, 140), ('任务场景', '大型战舰', 141, 145), ('任务场景', '巡逻舰', 146, 14

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', 'BlockⅠ型“标准”-6(SM-6)导弹', 19, 40), ('试验要素', '反舰试验', 45, 49), ('性能指标', '水面目标', 61, 65), ('性能指标', '超视距', 67, 70), ('性能指标', '反介入/区域拒止威胁', 84, 94)]


俄罗斯卡-52“短吻鳄”新型全天候侦察攻击直升机在其远东地区进行了飞行试验?根据任务安排,卡-52直升机在持续1小时的飞行试验中演示了上升?盘旋?转向等基本的空中机动动作,以及平转?盘旋扫射和回转等特技飞行动作?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '上升', 67, 69), ('试验要素', '盘旋', 70, 72), ('试验要素', '转向', 73, 75), ('试验要素', '平转', 88, 90), ('试验要素', '盘旋扫射', 91, 95), ('试验要素', '回转', 96, 98)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '上升', 67, 69), ('性能指标', '盘旋', 70, 72), ('性能指标', '转向', 73, 75), ('性能指标', '平转', 88, 90), ('性能指标', '盘旋扫射', 91, 95), ('性能指标', '回转', 96, 98)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('系统组成', '舰载作战控制系统', 128, 136)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '“联合濒海训练演习16”', 1, 13), ('试验要素', '“哈利法克斯”级护卫舰“范库弗峰”号', 20, 38), ('试验要素', '岸基靶标', 43, 47), ('试验要素', '“鱼叉”BlockⅡ型面面导弹', 53, 68), ('试验要素', '升级后舰载作战控制系统', 125, 136)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '“联合濒海训练演习16”', 1, 13), ('试验要素', '“哈利法克斯”级护卫舰“范库弗峰”号', 20, 38), ('试验要素', '岸基靶标', 43, 47), ('试验要素', '“鱼叉”BlockⅡ型面面导弹', 53, 68), ('系统组成', '舰载作战控制系统', 128, 136)]


美国陆军对一体化防空反导作战指挥系统进行了交战飞行试验?试验中,陆军联合诺斯罗普·格鲁曼公司把“哨兵”和“爱国者”雷达获得的数据成功用在了“爱国者”-3防空导弹系统的作战指挥控制系统上?此次试验验证了一体化防空反导作战指挥系统利用传感器和防空导弹对目标进行识别?追踪?打击和击毁的能力?飞行试验还包括对海军陆战队战术空中作战模块的试验,该模块用于联合作战指挥控制系统的态势感知?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('任务场景', '航天发射场', 140, 145), ('性能指标', '实际发射能力', 148, 154)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '“联盟”-2.1a运载火箭', 27, 40), ('试验要素', '“伏尔加”上面级', 41, 49), ('试验要素', '“MVL-300”科研卫星', 62, 75), ('试验要素', '“AIST-2D”科研卫星', 90, 103), ('试验要素', '“SamSat-218/D”纳星', 104, 120), ('试验要素', '预定轨道', 122, 126), ('任务场景', '俄罗斯', 157, 160), ('任务场景', '他国发射场', 163, 168)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
pre_label: [('试验要素', '“联盟”-2.1a运载火箭', 27, 40), ('试验要素', '“伏尔加”上面级', 41, 49), ('试验要素', '“MVL-300”科研卫星', 62, 75), ('试验要素', '“AIST-2D”科研卫星', 90, 103), ('试验要素', '“SamSat-218/D”纳星', 104, 120), ('试验要素', '预定轨道', 122, 126), ('任务场景', '航天发射场', 140, 145), ('性能指标', '实际发射能力', 148, 154), ('任务场景', '俄罗斯', 157, 160), ('

pre_label: [('试验要素', '“阿贾克斯”装甲战斗车', 11, 22), ('试验要素', '实弹射击试验', 36, 42), ('试验要素', '40毫米机关炮', 51, 58), ('试验要素', '并列机枪', 59, 63), ('试验要素', '遥控射击', 64, 68), ('试验要素', '遥控方式', 72, 76), ('性能指标', '静止', 77, 79), ('试验要素', '步兵战车', 80, 84), ('试验要素', '火炮', 87, 89), ('试验要素', '机枪射击试验', 90, 96)]


巴基斯坦海军近日称其在阿拉伯海的1处海域,试射了1枚岸基反舰导弹“扎布”,准确命中了1个水面目标?巴基斯坦海军参谋长穆罕默德·扎考拉未透露“扎布”岸基反舰导弹的具体细节,只是称在试验结束后巴海军将列装该型导弹,并称该型导弹将融入巴基斯坦现有的海岸防御体系,提高其区域拒止能力?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('任务场景', '区域拒止能力', 131, 137)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '区域拒止能力', 131, 137)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
true_label: [('试验要素', '岸基反舰导弹“扎布”', 26, 36), ('试验要素', '水面目标', 44, 48), ('任务场景', '巴基斯坦', 114, 118), ('任务场景', '海岸防御体系', 121, 127), ('任务场景', '区域拒止能力', 131,

## 测试集结果

In [40]:
from pathlib import Path

In [44]:
test_data_dir = Path('../data/fold0/test/')
with open(test_data_dir/'sentences.txt', 'r', encoding='utf-8') as f_sen,\
    open(test_data_dir/'tags.txt', 'r', encoding='utf-8') as f_tag:
    sentences = [line.strip().split(' ') for line in f_sen]
    tags = [line.strip().split(' ') for line in f_tag]

In [45]:
ex1_dir = Path('../ensemble/ex68')
import json
with open(ex1_dir/'ex68.json', 'r', encoding='utf-8') as f:
    pre = json.load(f)
y_pre = []
for idx in range(1, len(pre)+1):
    text = ''.join(sentences[idx-1])
    entities = pre[f'test_{idx}.json']
    sample = []
    for entity in entities:
        entity_con = text[entity['start_pos']-1:entity['end_pos']]
        sample.append((entity['label_type'], entity_con, entity['start_pos'], entity['end_pos']))
    y_pre.append(sample)

In [46]:
for sen, pre in zip(sentences, y_pre):
        print("".join(sen))
        print('-*-'*40)
        print(sorted(pre, key=lambda p: p[2]))
        print('\n')

﻿RT-2PM2弹道导弹的主要改进是在分系统应用了更先进的固体发动机、具有特殊弹道的弹头、命中精度更高的制导系统以及快速发射等新技术成果，战术技术性能指标明显提高。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeffRT-2PM2弹道导弹', 1, 12), ('系统组成', '分系统', 20, 22), ('系统组成', '固体发动机', 30, 34), ('系统组成', '特殊弹道', 38, 41), ('性能指标', '特殊弹道', 38, 41), ('性能指标', '弹道', 40, 41), ('系统组成', '弹头', 43, 44), ('性能指标', '命中精度', 46, 49), ('系统组成', '制导系统', 53, 56), ('性能指标', '快速发射', 59, 62), ('系统组成', '快速发射', 59, 62), ('性能指标', '战术技术性能', 70, 75), ('性能指标', '战术技术性能指标', 70, 77)]


高精确系统有限公司已完成了✈“杜香✈”新型防空导弹系统试验,即将进入列装阶段?“杜香✈”新型近程防空导弹系统已研发10余年,具备最新型激光制导系统,主要用于打击高精确?低空飞行目标,包括无人机?飞机?直升机和巡航导弹等?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“杜香✈”新型防空导弹系统', 15, 27), ('试验要素', '“杜香✈”新型近程防空导弹系统', 40, 54), ('系统组成', '新型激光制导系统', 66, 73), ('任务场景', '高精确', 81, 83), ('性能指标', '高精确', 81, 83), ('任务场景', '低空飞行目标', 85, 90), ('任务场景', '无人机', 94, 96), ('任

[('试验要素', '“僚机项目”', 50, 55), ('任务场景', '士兵', 68, 69), ('试验要素', '自主驾驶系统', 73, 78), ('试验要素', '高机动多用途轮式车', 87, 95), ('试验要素', '自主车辆', 109, 112), ('试验要素', '有人车辆', 117, 120), ('系统组成', '762毫米机枪', 137, 143), ('试验要素', '陆军士兵', 157, 160)]


“天基红外系统”(SBIRS)的新型地面系统Block✈10增量2进入试验阶段,该系统可同时对包括“国防支援计划”(DSP)卫星?SBIRS地球同步轨道卫星?SBIRS大椭圆轨道载荷在内的全部预警卫星星座/载荷进行控制?Block✈10增量2系统将SBIRS的操作控制集成至一个主要任务控制站及一个备份控制站,大幅简化控制流程,提高运行效率?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“天基红外系统”(SBIRS)', 1, 15), ('试验要素', '新型地面系统Block✈10增量2', 17, 33), ('试验要素', '“国防支援计划”(DSP)卫星', 50, 64), ('试验要素', 'SBIRS地球同步轨道卫星', 66, 78), ('试验要素', 'SBIRS大椭圆轨道载荷', 80, 91), ('任务场景', '预警卫星星座/载荷', 97, 105), ('试验要素', '预警卫星星座/载荷', 97, 105), ('性能指标', '操作控制', 131, 134), ('试验要素', '主要任务控制站', 140, 146), ('系统组成', '主要任务控制站', 140, 146), ('系统组成', '备份控制站', 150, 154), ('试验要素', '备份控制站', 150, 154), ('性能指标', '控制流程', 160, 163), ('性能指标', '运行效率', 167, 170)]


俄罗斯机械制造科学生产联合体在现役导弹“缟玛瑙”基础

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'DDG-85', 3, 8), ('试验要素', '伯克Flight✈2A', 12, 22), ('试验要素', '密集阵近防武器系统', 25, 33), ('系统组成', '密集阵近防武器系统', 25, 33), ('系统组成', '烟囱', 66, 67), ('系统组成', '埋入式设计', 76, 80), ('系统组成', '埋入式', 76, 78), ('系统组成', '顶端排气筒', 83, 87), ('系统组成', '烟囱结构', 90, 93), ('系统组成', '上层结构', 98, 101), ('性能指标', '雷达截面积', 112, 116), ('系统组成', '船舷甲板', 136, 139), ('试验要素', '三联装MK-32鱼雷发射器', 141, 153), ('系统组成', '三联装MK-32鱼雷发射器', 141, 153), ('系统组成', '机库', 157, 158), ('系统组成', '机库顶部垂直发射器', 157, 165), ('系统组成', '垂直发射器', 161, 165), ('试验要素', '鱼雷库', 174, 176), ('任务场景', '鱼雷库', 174, 176), ('性能指标', '鱼雷再装填', 203, 207), ('任务场景', '鱼雷再装填', 203, 207)]


美国陆军空间与导弹防御司令部于近期在新墨西哥州的白沙导弹靶场对“先进试验高能资产”✈(称“雅典娜”)系统样机进行了测试,验证了“雅典娜”激光武器系统应对无人机威胁的关键杀伤能力.测试期间,配备传感器,软件和专业光学系统的30千瓦“雅典娜”激光武器系统样机,利用先进的波束控制技术和光纤激光器对5架翼展为3.3米的“放逐者”无人机系统实施攻击.
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

[('试验要素', 'P5试车台', 16, 20), ('试验要素', '“火神”2.1(Vulcain2.1)发动机', 26, 47), ('试验要素', '点火试验', 49, 52), ('试验要素', '“阿里安”-6火箭芯一级', 67, 78), ('任务场景', '主动力', 80, 82), ('试验要素', '飞行包线', 92, 95), ('系统组成', '推力', 97, 98), ('系统组成', '混合比', 100, 102), ('系统组成', '推进剂', 104, 106), ('试验要素', '“阿里安”-6上面级Vinci发动机', 120, 137), ('试验要素', '点火能力演示验证', 185, 192)]


NASA表示,为火星任务设计的小型核动力系统✈“Kilopower✈”首次试验取得成功?Kilopower系统的目标是为未来空间任务提供能源,并可能为火星和其他行星表面任务提供能源?测试开始于2017年11月,在能源部的内华达州国家安全场开展?官方表示,系统的全面测试将于2018年3月进行?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '火星任务', 9, 12), ('试验要素', '小型核动力系统✈“Kilopower✈”', 16, 35), ('任务场景', '空间任务', 63, 66), ('任务场景', '能源', 69, 70), ('任务场景', '火星', 76, 77), ('任务场景', '行星表面任务', 81, 86)]


﻿美军此前的防空和反导导弹一般都采用“高能炸药破片全向飞散”的杀伤方式，往往只能实现所谓的“任务破坏”而非“导弹破坏”，即仅使来袭导弹偏离原定轨道，目标弹头内的爆炸物或生化战剂仍会散落地面造成损伤。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[(

[('试验要素', '俄亥俄级内布拉斯加号弹道导弹潜艇（SSBN-739）', 6, 31), ('系统组成', '战斗部', 44, 46), ('试验要素', '三叉戟II(D5)导弹', 48, 58), ('性能指标', '寿命', 99, 100), ('试验要素', '指挥官评估试验(CET)', 139, 150), ('性能指标', '预期性能', 180, 183), ('性能指标', '持续可靠性', 194, 198), ('性能指标', '准确性', 200, 202), ('性能指标', '针对性', 221, 223)]


﻿澳大利亚政府与英国超电子（Ultra✈Electronics）公司签约，由该公司为霍巴特级提供先进整合声纳系统，称为多重静态可变深度声纳（MMVDSS），整套系统包括一座舰首主/被动声纳和拖曳阵列声纳，由一套具备方向分辨能力的被动低频拖曳阵列声纳、高频主动声纳、专门警戒来袭鱼雷的被动声纳组成。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '霍巴特级', 43, 46), ('试验要素', '整合声纳系统', 51, 56), ('试验要素', '多重静态可变深度声纳（MMVDSS）', 60, 77), ('系统组成', '舰首主/被动声纳', 87, 94), ('系统组成', '拖曳阵列声纳', 96, 101), ('性能指标', '方向分辨能力', 108, 113), ('系统组成', '被动低频拖曳阵列声纳', 115, 124), ('系统组成', '高频主动声纳', 126, 131), ('任务场景', '来袭鱼雷', 137, 140), ('性能指标', '来袭鱼雷', 137, 140), ('系统组成', '被动声纳', 142, 145)]


日本和美国联合开发的“标准-3”Block✈2A导弹将于2017年春季完成研制,2018年开始部署;日本海上自卫队将为其两艘新型导弹驱逐舰装备“宙斯盾”基线9作战系统,这两艘战舰预计于2020年和2021年服役;计划利用5年时间对“

...

[('试验要素', '塔尔瓦（Talwar）级护卫舰', 5, 19), ('试验要素', '布拉莫斯舰射型巡航导弹', 26, 36), ('试验要素', '布拉莫斯导弹', 77, 82), ('任务场景', '俱乐部-N导弹', 89, 95), ('试验要素', '俱乐部-N导弹', 89, 95), ('系统组成', '发射管', 120, 122), ('系统组成', '垂直发射系统', 124, 129), ('系统组成', '集成舰载控制单元', 131, 138)]


﻿“爱国者”－3系统（PAC-3）最大改进之处便是采用新型的增程拦截弹（ERINT）。PAC－3虽然作战距离、高度等指标比PAC－2有所下降，但对高威胁目标的拦截效果却大大增强。作为第一种不依靠爆炸战斗部来摧毁目标的拦截弹，PAC－3/ERINT将通过直接撞击的方式摧毁位于大气层内的目标，而且每个火力单元预备8个层弹发射架（每个发射架上备有16枚ERINT弹），火力密度大大增强。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“爱国者”－3系统（PAC-3）', 2, 17), ('试验要素', '爱国者”－3系统（PAC-3）', 3, 17), ('试验要素', '增程拦截弹（ERINT）', 31, 42), ('系统组成', '增程拦截弹（ERINT）', 31, 42), ('性能指标', '作战距离', 51, 54), ('性能指标', '高度', 56, 57), ('性能指标', '高威胁目标', 74, 78), ('性能指标', '拦截效果', 80, 83), ('系统组成', '爆炸战斗部', 98, 102), ('性能指标', '直接撞击', 127, 130), ('性能指标', '大气层内', 138, 141), ('性能指标', '大气层', 138, 140), ('任务场景', '大气层', 138, 140), ('系统组成', '火力单元', 150, 153), ('试验要素', '火力单元', 150, 153), (

[('试验要素', '\ufeffLGM-30G弹道导弹发动机', 1, 15), ('系统组成', '潜入式固定单喷管', 18, 25), ('系统组成', '潜入式固定单喷管设计', 18, 27), ('系统组成', '单轴摆动喷管', 34, 39), ('系统组成', '喷管本体材料', 41, 46), ('系统组成', '7075-T73铝合金', 48, 58), ('系统组成', '推力矢量控制', 60, 65), ('系统组成', '过氯酸锶液体喷射系统', 68, 77), ('系统组成', '第三级', 80, 82), ('系统组成', '推力终止系统', 87, 92), ('系统组成', '燃气旋转控制系统', 98, 105)]


近日，国防高级研究计划局首次公布OFFSET项目第二次试验的细节信息。试验在跨越两个城市街区的复杂场景中进行，无人机和地面机器人编队执行了各种任务，包括定位和隔离模拟的市政厅，并定位建筑内部目标，然后为建筑提供安保。同时全程保持对周围环境的态势感知。试验持续了30分钟。本次试验是6次计划冲刺中的第2次，计划6个月后开展下一次试验，重点测试人-蜂群编队方案，已经确定的参与者包括凯斯西储大学、查尔斯河研究公司和西北大学。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'OFFSET项目', 17, 24), ('试验要素', '城市街区', 43, 46), ('试验要素', '复杂场景', 48, 51), ('试验要素', '无人机', 56, 58), ('试验要素', '地面机器人', 60, 64), ('任务场景', '市政厅', 85, 87), ('试验要素', '市政厅', 85, 87), ('试验要素', '建筑内部目标', 92, 97), ('任务场景', '安保', 106, 107), ('试验要素', '安保', 106, 107), ('性能指标', '周围环境', 116, 119), ('性能指标', '态势感知', 121, 124), ('试验要素',

[('试验要素', '\ufeff伯克级', 1, 4), ('试验要素', 'MK-41垂直发射系统', 12, 22), ('系统组成', 'MK-41垂直发射系统', 12, 22), ('性能指标', '载弹量', 24, 26), ('试验要素', '海上再装填起重机', 38, 45), ('系统组成', '海上再装填起重机', 38, 45), ('试验要素', '标准SM-2导弹', 50, 57), ('试验要素', 'SM-2导弹', 52, 57), ('性能指标', '重', 62, 62), ('任务场景', '战斧导弹', 64, 67), ('试验要素', '战斧导弹', 64, 67), ('试验要素', '伯克Flight✈1/1A/2', 86, 100), ('系统组成', '舰尾', 102, 103), ('系统组成', '直升机甲板', 106, 110), ('系统组成', '机库', 113, 114), ('系统组成', 'SQQ-89（V）4/6反潜作战系统', 139, 156), ('试验要素', 'SQQ-89（V）4/6反潜作战系统', 139, 156), ('系统组成', 'SQS-53C舰首声呐', 160, 170), ('系统组成', 'SQR-19线列阵声呐', 172, 182), ('系统组成', 'SQQ-28直升机数据链', 185, 196), ('系统组成', 'MK-116Mod7反潜火控系统', 200, 215), ('试验要素', 'MK-116Mod7反潜火控系统', 200, 215)]


北方舰队不久前部署了“舞会”岸防反舰导弹系统，并将于秋季进行试射。“舞会”岸防反舰导弹系统最终将完全代替“边界”导弹系统进入北方舰队服役，主要用于打击150千米的海上目标。“舞会”岸防反舰导弹系统装配kh-35亚声速低空反舰导弹，能够摧毁排水量达5000吨的船只。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“舞会”岸防反舰导弹系统', 11, 22), ('试验要素

从2月底到4月12日,美国陆军联合海军陆战队在海军陆战队空中作战中心加利福尼亚东部的莫哈韦沙漠开始对新型✈“联合轻型战术车辆✈”(JLTV)开展试验?该试验收集到的数据将用于解决✈“联合轻型战术车辆✈”在预期环境中的作战有效性?适用性和生存能力等方面的问题?
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“联合轻型战术车辆✈”(JLTV)', 54, 70), ('性能指标', '预期环境', 103, 106), ('性能指标', '作战有效性', 109, 113), ('性能指标', '适用性', 115, 117), ('性能指标', '生存能力', 119, 122)]


﻿通过提高雷达信息设备性能和优化雷达信号处理方法，安泰-2500对付小反射截面高速弹道目标的能力进一步提高。一个安泰-2500地空导弹营可在1000～2000平方千米范围内拦截各种型号的弹道导弹，在12500平方千米范围内摧毁敌航空兵器的袭击。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('系统组成', '雷达信息设备', 6, 11), ('系统组成', '雷达信号处理方法', 17, 24), ('试验要素', '雷达信号处理方法', 17, 24), ('试验要素', '安泰-2500', 26, 32), ('性能指标', '小反射截面高速弹道目标', 35, 45), ('试验要素', '安泰-2500地空导弹营', 57, 68), ('试验要素', '地空导弹营', 64, 68), ('性能指标', '弹道导弹', 94, 97), ('任务场景', '弹道导弹', 94, 97), ('任务场景', '敌航空兵器', 114, 118)]


﻿LGM-30G弹道导弹的制导与控制采用NS-20系统，主要由陀螺稳定平台、数字计算机、放大器组件、电子控制装置和

[('试验要素', '\ufeff世宗大王级驱逐舰(KDX-3)', 1, 16), ('性能指标', '反舰与远程对地打击火力', 22, 32), ('性能指标', '对地', 40, 41), ('性能指标', '对地攻击', 40, 43), ('系统组成', '舰炮', 45, 46), ('试验要素', '舰炮', 45, 46), ('试验要素', '对地/反舰巡航导弹', 48, 56), ('系统组成', '对地/反舰巡航导弹', 48, 56), ('试验要素', 'MK-45✈Mod4✈五英寸62倍径舰炮', 64, 83), ('系统组成', 'MK-45✈Mod4✈五英寸62倍径舰炮', 64, 83), ('任务场景', 'ERGM制导炮弹', 88, 95), ('试验要素', 'ERGM制导炮弹', 88, 95), ('任务场景', '沿岸目标', 105, 108), ('系统组成', '八联装MK-41垂直发射单元', 114, 127), ('系统组成', '韩国垂直发射系统', 140, 147), ('系统组成', '垂直发射系统', 142, 147), ('系统组成', '垂直发射器', 151, 155), ('性能指标', '发射管数', 158, 161), ('任务场景', '伯克级', 171, 173), ('任务场景', '金刚级', 175, 177), ('试验要素', '提康德罗加级巡洋舰', 190, 198), ('任务场景', '提康德罗加级巡洋舰', 190, 198)]


﻿老式俄制陆基机动导弹受体积限制只能携带单弹头，严重限制了打击能力，RS-24可以做到携带至少3枚分导弹头打击不同的目标，机动发射加上多弹头这两种能力，让RS-24更加难对付。RS-24的战略意义在于击毁敌人领土上的军事和有经济潜力的设施。该导弹号称可对付在梯次配置反导系统保护下的目标。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '陆基机动导弹', 6, 11), ('性能指标', '体积', 13, 

两结果对比

In [47]:
ex1_dir = Path('../ensemble/old_fusion_results/ex20')
import json
with open(ex1_dir/'fusion_submit.json', 'r', encoding='utf-8') as f:
    pre = json.load(f)
y_true = []
for idx in range(1, len(pre)+1):
    text = ''.join(sentences[idx-1])
    entities = pre[f'test_{idx}.json']
    sample = []
    for entity in entities:
        entity_con = text[entity['start_pos']-1:entity['end_pos']]
        sample.append((entity['label_type'], entity_con, entity['start_pos'], entity['end_pos']))
    y_true.append(sample)

In [49]:
for sen, pre, true in zip(sentences, y_pre, y_true):
        print("".join(sen))
        print('-*-'*40)
        # 真实标签里有而预测结果里没有的（影响Recall）
        print(sorted(list(set(true).difference(set(pre))), key=lambda p: p[2]))
        print('-*-'*40)
        # 预测结果里有而真实标签里没有的（影响Precision）
        print(sorted(list(set(pre).difference(set(true))), key=lambda p: p[2]))
        print('-*-'*40)
        print(sorted(true, key=lambda p: p[2]))
        print('-*-'*40)
        print(sorted(pre, key=lambda p: p[2]))
        print('\n')

﻿RT-2PM2弹道导弹的主要改进是在分系统应用了更先进的固体发动机、具有特殊弹道的弹头、命中精度更高的制导系统以及快速发射等新技术成果，战术技术性能指标明显提高。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '快速发射', 59, 62), ('性能指标', '战术技术性能', 70, 75)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeffRT-2PM2弹道导弹', 1, 12), ('系统组成', '分系统', 20, 22), ('系统组成', '固体发动机', 30, 34), ('系统组成', '特殊弹道', 38, 41), ('性能指标', '特殊弹道', 38, 41), ('性能指标', '弹道', 40, 41), ('系统组成', '弹头', 43, 44), ('性能指标', '命中精度', 46, 49), ('系统组成', '制导系统', 53, 56), ('系统组成', '快速发射', 59, 62), ('性能指标', '战术技术性能指标', 70, 77)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeffRT-2PM2弹道导弹', 1, 12), ('系统组成', '分系统', 20, 22), ('系统组成', '固体

[('试验要素', '短程(SRBM)', 33, 40)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“宙斯盾”弹道导弹防御系统', 1, 13), ('任务场景', '战区级弹道导弹', 24, 30), ('任务场景', '短程(SRBM)', 33, 40), ('任务场景', '准中程(MRBM)', 42, 50), ('试验要素', '准中程(MRBM)', 42, 50), ('试验要素', '中程弹道导弹(IRBM)', 52, 63), ('任务场景', '中程弹道导弹(IRBM)', 52, 63), ('系统组成', '雷达', 90, 91), ('试验要素', '跟踪数据', 96, 99), ('任务场景', '洲际导弹导弹', 112, 117), ('任务场景', '防御系统', 119, 122)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '“宙斯盾”弹道导弹防御系统', 1, 13), ('任务场景', '战区级弹道导弹', 24, 30), ('任务场景', '短程(SRBM)', 33, 40), ('试验要素', '短程(SRBM)', 33, 40), ('任务场景', '准中程(MRBM)', 42, 50), ('试验要素', '准中程(MRBM)', 42, 50), ('试验要素', '中程弹道导弹(IRBM)', 52, 63), ('任务场景', '中程弹道导弹(IRBM)', 52, 63), ('系统组成', '雷达', 90, 91), ('试验要素', '跟踪数据', 96, 99), ('任务场景', '洲际导弹导弹', 112, 117), ('任务场景', '防御系统', 119, 122)]


﻿在反潜、电战系统上，舰体声纳使用了性能较差但适合浅海操作的中

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '大规模杀伤性弹头', 56, 63), ('系统组成', '小型炸弹', 101, 104)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹', 1, 4), ('试验要素', '巡航导弹', 6, 9), ('系统组成', '常规和非常规弹头', 13, 20), ('系统组成', '常规弹头', 22, 25), ('系统组成', '弹药', 28, 29), ('性能指标', '杀伤效果', 35, 38), ('系统组成', '核', 48, 48), ('系统组成', '生物', 50, 51), ('系统组成', '化学', 53, 54), ('系统组成', '大规模杀伤性弹头', 56, 63), ('任务场景', '设备', 70, 71), ('系统组成', '非致命弹头', 75, 79), ('系统组成', '弹头', 96, 97), ('试验要素', '小型炸弹', 101, 104), ('性能指标', '高度', 109, 110)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹', 1, 4), ('试验要素', '巡航导弹', 6, 9), ('系统组成', '常规和非常规弹头', 13, 20), ('系统组成', '弹药', 28, 29), ('性能指标', '杀伤效果', 35, 38), ('系统组成', '核', 48, 48), ('系统组成', '生物', 50, 51), ('系统组成', '化学', 53

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'KIMF型碳-碳复合材料', 7, 18), ('任务场景', '“锆石”高超声速导弹弹体', 28, 39), ('试验要素', '“锆石”高超声速导弹弹体', 28, 39), ('性能指标', '三维增强', 47, 50), ('系统组成', '碳-碳复合材料', 52, 58), ('性能指标', '耐烧蚀性', 63, 66), ('性能指标', '弹性强度', 69, 72), ('性能指标', '耐热', 75, 76), ('试验要素', 'R-29RM潜射弹道导弹弹头', 86, 99), ('任务场景', 'R-29RM潜射弹道导弹弹头', 86, 99), ('性能指标', '结构重量', 108, 111), ('性能指标', '隐身性', 120, 122), ('性能指标', '敌方雷达', 127, 130), ('任务场景', '敌方雷达', 127, 130)]


巴西航空工业公司计划于2020年12月完成MTC-300远程陆基战术巡航导弹的试验，由巴西陆军的MK3M和MK6多管火箭炮系统发射，每辆发射车可携带两个导弹发射箱，每个发射箱可装载一枚MTC-300导弹。MTC-300远程陆基战术巡航导弹重约1100千克，弹长5.43米，弹径450毫米，巡航速度1044公里/小时（马赫数0.85），射程超过300千米，圆概率误差（CEP）为30米，采用折叠翼和两组十字形尾翼，装备亚声速发动机和两级固体火箭助推器，使用GPS和惯性制导系统，战斗部为重200千克的高爆杀伤/反装甲子母战斗部。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'MK3M', 49, 52), ('试验要素', 'MK6多管火箭炮系统', 54, 63), ('系统组成', '高爆杀伤/反装

[('试验要素', '战斗部', 40, 42)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '高超声速武器', 19, 24), ('系统组成', '杀伤力增强弹药(LEO)战斗部', 122, 136), ('试验要素', '破片设计', 146, 149), ('系统组成', '建模技术', 151, 154)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('任务场景', '高超声速武器', 19, 24), ('试验要素', '3D打印战斗部', 26, 32), ('性能指标', '重', 34, 34), ('试验要素', '战斗部', 40, 42), ('试验要素', '地下掩体', 64, 67), ('性能指标', '地面', 75, 76), ('试验要素', '冲击波', 80, 82), ('试验要素', '杀伤力增强弹药(LEO)战斗部', 122, 136), ('任务场景', '破片设计', 146, 149), ('试验要素', '建模技术', 151, 154)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '高超声速武器', 19, 24), ('任务场景', '高超声速武器', 19, 24), ('试验要素', '3D打印战斗部', 26, 32), ('性能指标', '重', 34, 34), ('试验要素', '地下掩体', 64, 67), ('性能指标', '地面', 75, 76), ('试验要素', '冲击波', 80, 82), ('试验要素', '杀伤力增强弹药(LEO)战斗部',

...

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('任务场景', '超声速和高超声速巡航导弹', 58, 69), ('试验要素', '超声速和高超声速巡航导弹', 58, 69), ('试验要素', '传统弹道导弹', 73, 78)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '弹道导弹', 9, 12), ('性能指标', '巡航导弹', 14, 17), ('性能指标', '超声速', 58, 60), ('性能指标', '高超声速', 62, 65)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹', 9, 12), ('试验要素', '巡航导弹', 14, 17), ('试验要素', '高超声速滑翔飞行器(HGVs)', 31, 45), ('试验要素', '无动力巡航导弹', 47, 53), ('试验要素', '超声速和高超声速巡航导弹', 58, 69), ('任务场景', '超声速和高超声速巡航导弹', 58, 69), ('试验要素', '传统弹道导弹', 73, 78), ('系统组成', '大型火箭助推器', 80, 86)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '弹道导弹', 9, 12), ('性能指标', '弹道导弹', 9, 12), ('试验要素', '巡航导弹', 14, 1

﻿S-400防空导弹为了确保研发工艺的继承性和最大限度节省研制经费，除了装备l部新型超视距雷达和3种新型防空导弹外，基本上沿用了S-300防空导弹系统的指挥控制系统、导弹发射系统和战斗单元编成。并首次采用了三种新型导弹和机动目标搜索系统，可以对付各种作战飞机、空中预警机、战役战术导弹及其他精确制导武器既能承担传统的空中防御任务，又能执行非战略性的导弹防御任务。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('性能指标', '继承性', 20, 22)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', 'S-400防空导弹', 2, 10), ('试验要素', '新型防空导弹', 51, 56)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeffS-400防空导弹', 1, 10), ('性能指标', '继承性', 20, 22), ('系统组成', '新型超视距雷达', 41, 47), ('系统组成', '新型防空导弹', 51, 56), ('试验要素', 'S-300防空导弹系统', 65, 75), ('系统组成', '指挥控制系统', 77, 82), ('系统组成', '导弹发射系统', 84, 89), ('系统组成', '战斗单元', 91, 94), ('系统组成', '战斗单元编成', 91, 96), ('系统组成', '新型导弹', 106, 109), ('系统组成', '机动目标搜索系统', 111, 118), ('任务场景', '作战飞机', 126, 129), ('任务场景', '空中预警机', 13

[('试验要素', '\ufeff世宗大王级驱逐舰(KDX-3)', 1, 16), ('性能指标', '反舰与远程对地打击火力', 22, 32), ('性能指标', '对地', 40, 41), ('性能指标', '对地攻击', 40, 43), ('系统组成', '舰炮', 45, 46), ('试验要素', '舰炮', 45, 46), ('试验要素', '对地/反舰巡航导弹', 48, 56), ('系统组成', '对地/反舰巡航导弹', 48, 56), ('试验要素', 'MK-45✈Mod4✈五英寸62倍径舰炮', 64, 83), ('系统组成', 'MK-45✈Mod4✈五英寸62倍径舰炮', 64, 83), ('任务场景', 'ERGM制导炮弹', 88, 95), ('试验要素', 'ERGM制导炮弹', 88, 95), ('任务场景', '沿岸目标', 105, 108), ('系统组成', '八联装MK-41垂直发射单元', 114, 127), ('系统组成', '韩国垂直发射系统', 140, 147), ('系统组成', '垂直发射系统', 142, 147), ('系统组成', '垂直发射器', 151, 155), ('性能指标', '发射管数', 158, 161), ('任务场景', '伯克级', 171, 173), ('任务场景', '金刚级', 175, 177), ('试验要素', '提康德罗加级巡洋舰', 190, 198), ('任务场景', '提康德罗加级巡洋舰', 190, 198)]


﻿老式俄制陆基机动导弹受体积限制只能携带单弹头，严重限制了打击能力，RS-24可以做到携带至少3枚分导弹头打击不同的目标，机动发射加上多弹头这两种能力，让RS-24更加难对付。RS-24的战略意义在于击毁敌人领土上的军事和有经济潜力的设施。该导弹号称可对付在梯次配置反导系统保护下的目标。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '老式俄制陆基机动导弹', 2, 11), ('性能指标', '目标', 

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeffS-400防空导弹武器系统（SA-21/萨姆-21）', 1, 27), ('任务场景', '陆基移动式中远程防空导弹系统', 33, 46), ('试验要素', '陆基移动式中远程防空导弹系统', 33, 46), ('任务场景', '地对空导弹系统', 58, 64), ('任务场景', '超低空', 70, 72), ('性能指标', '超低空', 70, 72), ('任务场景', '高空', 74, 75), ('性能指标', '高空', 74, 75), ('任务场景', '近距离', 77, 79), ('性能指标', '近距离', 77, 79), ('任务场景', '超远程', 81, 83), ('性能指标', '超远程', 81, 83), ('任务场景', '全空域', 85, 87), ('性能指标', '全空域', 85, 87), ('性能指标', '密集多目标空袭', 90, 96), ('任务场景', '密集多目标空袭', 90, 96), ('任务场景', '第五代战斗机', 101, 106), ('试验要素', '第五代战斗机', 101, 106), ('性能指标', '干扰与对抗环境', 111, 117), ('试验要素', '干扰与对抗环境', 111, 117), ('性能指标', '反战术和中程弹道导弹', 128, 137), ('性能指标', '反战术', 128, 130), ('性能指标', '战术和中程弹道导弹', 129, 137), ('性能指标', '中程弹道导弹', 132, 137)]


﻿“爱国者”-3防空导弹武器系统交战控制站，采用人机交互的方式，可以由计算机辅助进行目标识别和优先级排序，也可以由交战控制站和计算机完全自主控制整个作战。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

[('试验要素', '\ufeff22350型护卫舰', 1, 10), ('任务场景', '中型防空导弹护卫舰', 20, 28), ('性能指标', '排水量', 30, 32), ('系统组成', '主桅杆', 41, 43), ('系统组成', '四面固定式多功能相控阵雷达', 46, 58), ('系统组成', '固定式多功能相控阵雷达', 48, 58), ('试验要素', '28单元鲁道特导弹垂直发射装置', 63, 77), ('任务场景', '导弹', 80, 81), ('任务场景', '制导', 84, 85), ('性能指标', '制导', 84, 85), ('系统组成', '主桅杆', 87, 89), ('系统组成', '旋转式三维搜索相控阵雷达', 96, 107), ('系统组成', '130毫米舰炮', 115, 121), ('试验要素', '反舰导弹', 125, 128), ('系统组成', '鲁道特', 132, 134), ('系统组成', '垂直发射装置', 141, 146), ('试验要素', '垂直发射装置', 141, 146), ('试验要素', '红宝石', 155, 157), ('试验要素', '红宝石或布拉莫斯反舰导弹', 155, 166), ('试验要素', '布拉莫斯反舰导弹', 159, 166), ('性能指标', '配置布局', 179, 182), ('性能指标', '综合火力', 186, 189)]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('试验要素', '\ufeff22350型护卫舰', 1, 10), ('任务场景', '中型防空导弹护卫舰', 20, 28), ('试验要素', '中型防空导弹护卫舰', 20, 28), ('性能指标', '排水量', 30, 32), ('系统组成', '主桅杆', 41, 43), ('系统组成', '四面固定式多功能相控阵雷达', 46, 58), ('系统组成', '固定式多功能相控阵雷达', 48, 58), ('试验要素', '